# 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# [Step 0] 초기 설정

In [ ]:
!pip install fake_useragent

In [ ]:
import requests
import os
import json
import base64
import urllib3
import shutil
from PIL import Image
from fake_useragent import UserAgent
import time

api_url = 'http://ec2-13-209-17-240.ap-northeast-2.compute.amazonaws.com/'

login_url = 'user/login'
image_url = 'animal/alllist'
username = 'capstone'
password = 'capstone2023!'

dataset_path = os.getcwd() + '/drive/MyDrive/2023-2-Capstone-ML/dataset'
pet_img_path = dataset_path + '/pet_img'
pet_crop_path = dataset_path + '/pet_crop_img'

ua = UserAgent()

# [Step 1] 로그인 토큰 발급

In [ ]:
login_res = requests.post(url=api_url + login_url,
                          data={
                              "username": username,
                              "password": password,
                          })
print(login_res.json())
token = login_res.json()['data']['token']
token = 'Bearer ' + token

headers = {'Authorization': token}

# [Step 2] 사진 다운로드

In [ ]:
res = requests.get(url=api_url + image_url, headers=headers)

for data in res.json()['data']:
  animal_id = data['animal_id']
  if animal_id not in [ '17']:
    continue
  image_path = data['image']
  image_data = requests.get(api_url + image_path)
  file_name = image_path.split("/")[-1]

  # 폴더 생성
  dir_path = os.path.join(pet_img_path, animal_id)
  os.makedirs(dir_path, exist_ok=True)

  # 사진 저장
  file_path = os.path.join(dir_path, file_name)
  image = open(file_path, 'wb')
  image.write(image_data.content)
  image.close()

In [ ]:
print(os.listdir(pet_img_path))

# [Step 3] 객체 인식 후 사진 자르기

In [ ]:
def crop_api(image_file_path):
  open_api_url = "http://aiopen.etri.re.kr:8000/ObjectDetect"
  access_key = "d5032a3a-3b61-4dec-bf26-4a3c804d0691"
  image_type = "jpg"

  file = open(image_file_path, "rb")
  imageContents = base64.b64encode(file.read()).decode("utf8")
  file.close()

  requestJson = {
      "argument": {
          "type": image_type,
          "file": imageContents
      }
  }

  http = urllib3.PoolManager()
  http_response = requests.post(
      open_api_url,
      headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": access_key,
                 "User-Agent": ua.random},
      data=json.dumps(requestJson),
      timeout=60
  )

  if http_response.status_code != 200 :
    print(http_response.status_code)
    return -1

  print(http_response.json())
  return http_response.json()['return_object']['data']

In [ ]:
def save_crop_img(source, destination, crop_response):
  accept_class = ['dog', 'teddy bear']
  for object_class in crop_response:
    if object_class['class'] not in accept_class : continue

    shutil.copy(source, destination)
    img = Image.open(destination)
    img = img.convert('RGB')
    img_cropped = img.crop((int(object_class['x']),
                            int(object_class['y']),
                            int(object_class['x']) + int(object_class['width']),
                            int(object_class['y']) + int(object_class['height'])))
    img_cropped.show()
    img_cropped.save(destination)

    print(str(destination)  + "에 저장 완료")
    return 1

  print("일치하는 객체가 없습니다.")

In [ ]:
pet_dir = os.listdir(pet_img_path)
pet_dir = ['17', '18', '20']
for pet_id in pet_dir:
  pet_img_dir_path = os.path.join(pet_img_path, pet_id)
  pet_img_dir = os.listdir(pet_img_dir_path)
  pet_crop_dir_path = os.path.join(pet_crop_path, pet_id)
  os.makedirs(pet_crop_dir_path, exist_ok=True)

  for pet_img in pet_img_dir:
    try :
      target = os.path.join(pet_img_path, pet_id, pet_img)
      response = crop_api(target)

      if response == -1 :
        print("response 오류")
        continue

      save_point = os.path.join(pet_crop_path, pet_id, pet_img)
      save_crop_img(target, save_point, response)
    except Exception as e :
      print(e)
      print("넘어갈게유")
